In [17]:
import os
import pandas as pd

In [19]:
main_directory = "C:/Users/PC/Documents/ABC_2025/output_csvs"

In [21]:
activity_mapping = {str(i): i - 2805 for i in range(2806, 2815)}

In [23]:
def segment_data_by_time(df, segment_duration=3):
    segments = []
    start_time = df['Timestamp'].iloc[0]
    end_time = df['Timestamp'].iloc[-1]

    current_time = start_time

    while current_time + pd.Timedelta(seconds=segment_duration) <= end_time:
        segment = df[(df['Timestamp'] >= current_time) & (df['Timestamp'] < current_time + pd.Timedelta(seconds=segment_duration))]
        segments.append(segment)
        current_time += pd.Timedelta(seconds=segment_duration)

    return segments

In [25]:
def load_data(directory):
    all_segments = []
    for file_name in os.listdir(directory):
        if file_name.endswith('.csv'):
            try:
                file_path = os.path.join(directory, file_name)
                df = pd.read_csv(file_path)
                df.columns = ['MotionType', 'Timestamp', 'X', 'Y', 'Z']  

                df['Timestamp'] = pd.to_datetime(df['Timestamp'])
                
                activity_key = file_name[:4] 
                activity = activity_mapping.get(activity_key, None)

                df['Activity'] = activity
                df = df.drop(columns=['MotionType'])  
                df = df[['Activity', 'Timestamp', 'X', 'Y', 'Z']]  

                segments = segment_data_by_time(df, segment_duration=3)

                for segment in segments:
                    segment = segment.drop(columns=['Timestamp']) 
                    all_segments.append(segment)
                    
            except Exception as e:
                print(f"Error reading {file_name}: {e}")

    return all_segments

In [27]:
output_directory = os.path.join(main_directory, 'segmented_dt1')
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [29]:
all_segments = load_data(main_directory)

In [30]:
for i, segment in enumerate(all_segments):
    segment_file_name = f"segment_{i + 1}.csv"
    segment_file_path = os.path.join(output_directory, segment_file_name)
    segment.to_csv(segment_file_path, index=False)

print("Segment succeeded")


Segment succeeded
